In [1]:
import pandas as pd
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
from time import sleep
import os
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
import numpy as np

In [2]:
os.chdir('D:/MSC/NLP/NLP_ASS2/semeval-tweets/')
import twokenize
BASE_DIR = 'D:/MSC/NLP/NLP_ASS2/semeval-tweets/'

In [3]:
## Contractions
file = open(BASE_DIR + 'CONTRACTIONS.pkl', "rb")
CONTRACTION_MAP = pickle.load(file)
sleep(1)
del file
contraction = sorted(CONTRACTION_MAP, key=len, reverse=True)
c_re = re.compile('(%s)' % '|'.join(contraction))
def expandContractions(text, c_re=c_re):
    def replace(match):
        return CONTRACTION_MAP[match.group(0)]
    return c_re.sub(replace, text.lower())

## Emojis Regex
file = open(BASE_DIR + 'EMOJIS.pkl', "rb")
EMOJIS = pickle.load(file)
sleep(1)
del file
emojis = sorted(EMOJIS, key=len, reverse=True)
pattern = '|'.join(re.escape(u) for u in emojis)
regex = re.compile(pattern, re.U)

## Regex expression for Booster Increase and Decrease words
B_INC = ['exceptionally', 'substantially', 'considerable', 'considerably', 'particularly', 'tremendously', 'unbelievably',\
         'exceptional', 'absolutely', 'completely', 'enormously', 'especially', 'fabulously', 'incredible', 'incredibly',\
         'remarkably', 'thoroughly', 'tremendous', 'amazingly', 'decidedly', 'extremely', 'intensely', 'unusually',\
         'enormous', 'entirely', 'flipping', 'fracking', 'fricking', 'frigging', 'awfully', 'extreme', 'flippin',\
         'frackin', 'frickin', 'friggin', 'fucking', 'fugging', 'greatly', 'majorly', 'totally', 'utterly', 'deeply',\
         'effing', 'fuckin', 'fuggin', 'highly', 'hugely', 'purely', 'really', 'fully', 'hella', 'major', 'quite',\
         'total', 'utter', 'more', 'most', 'uber', 'very', 'so']
B_DEC = ['almost', 'barely', 'hardly', 'just enough', 'kind of', 'kinda', 'kindof', 'kind-of', 'less', 'little',\
         'marginal', 'marginally', 'occasional', 'occasionally', 'partly', 'scarce', 'scarcely', 'slight', 'slightly',\
         'somewhat', 'sort of', 'sorta', 'sortof', 'sort-of']
pattern_binc = '|'.join(re.escape(u) for u in B_INC)
pattern_bdec = '|'.join(re.escape(u) for u in B_DEC)

## Reading EXICON DICT 
file = open(BASE_DIR + 'VADER_LEXICONS_SCORE.pkl', "rb")
LEXICON_DICT = pickle.load(file)
sleep(1)
del file


file = open(BASE_DIR + 'POSITIVE.pkl', "rb")
POS_LIST = pickle.load(file)
sleep(1)
del file
POS_LIST = sorted(POS_LIST, key=len, reverse=True)
pattern_pos = '|'.join(re.escape(u) for u in POS_LIST)

file = open(BASE_DIR + 'NEGATIVE.pkl', "rb")
NEG_LIST = pickle.load(file)
sleep(1)
del file
NEG_LIST = sorted(NEG_LIST, key=len, reverse=True)
pattern_neg = '|'.join(re.escape(u) for u in NEG_LIST)

file = open(BASE_DIR + 'BAD.pkl', "rb")
BAD_LIST = pickle.load(file)
sleep(1)
del file
BAD_LIST = sorted(BAD_LIST, key=len, reverse=True)
pattern_bad = '|'.join(re.escape(u) for u in BAD_LIST)

In [4]:
def return_tok_val(sentence):
    pos_tok = []
    neg_tok = []
    tokens = twokenize.tokenizeRawTweetText(sentence)
    ln_all_tokens = len(tokens)
    for toke in tokens:
        val = LEXICON_DICT.get(toke)
        if val:
            if val >0:
                pos_tok.append(val)
            else:
                neg_tok.append(val)
    
    return tokens,ln_all_tokens,len(pos_tok),len(neg_tok),sum(pos_tok),sum(neg_tok)

def clean_data(data,col,re_inc_boostr,re_dec_boostr,re_pos,re_neg,re_bad):
    dataframe = data.copy()
    link_regex = re.compile(r'(?:ftp|https?|www|file)\.?:?[//|\\\\]?[\w\d:#@%/;$()~_?\+-=\\\&]+\.[\w\d:#@%/;$~_?\+-=\\\&]+')
    dataframe = dataframe.assign(**dict(zip([col,'num_link'], zip(*dataframe['text'].apply(lambda x: re.subn(link_regex,'LINK',x) ) ))))
    dataframe = dataframe.assign(**dict(zip([col,'num_usermention'], zip(*dataframe[col].apply(lambda x: re.subn(r'@[\w]*','USERMENTION',x)) ))))
    dataframe = dataframe.assign(**dict(zip([col,'num_hashtag'], zip(*dataframe[col].apply(lambda x: re.subn(r'#[\w]*','HASHTAG',x) ) ))))
    dataframe = dataframe.assign(**dict(zip([col,'num_emoji'], zip(*dataframe[col].apply(lambda x: re.subn(regex, lambda m: EMOJIS.get(m.group(), 'EMOJI') , x) )))))
    dataframe[col] = dataframe[col].apply(lambda x : re.sub(r'(.)\1{2,}', r'\1',x)) # make looong as long
    dataframe[col] = dataframe[col].apply(lambda x : expandContractions(x) ) # expand contracts
    dataframe[col] = dataframe[col].str.lower()
    dataframe['NUM_INC_BOOSTR'] = dataframe[col].apply(lambda x : len(re.findall(re_inc_boostr,x)) ) # Booster Increasing Words
    dataframe['NUM_DEC_BOOSTR'] = dataframe[col].apply(lambda x : len(re.findall(re_dec_boostr,x)) ) # Booster Increasing Words
    dataframe['NUM_POS_WORDS'] = dataframe[col].apply(lambda x : len(re.findall(re_pos,x)) ) # Number of Positive words
    dataframe['NUM_NEG_WORDS'] = dataframe[col].apply(lambda x : len(re.findall(re_neg,x)) ) # Number of Positive words
    dataframe['NUM_BAD_WORDS'] = dataframe[col].apply(lambda x : len(re.findall(re_bad,x)) ) # Number of Positive words
    dataframe = dataframe.assign(**dict(zip([col,'num_exclaim'], zip(*dataframe[col].apply(lambda x: re.subn(r"[!]",'',x)  ) ))))  
    dataframe = dataframe.assign(**dict(zip([col,'num_punct'], zip(*dataframe[col].apply(lambda x: re.subn(r"['\"“”‘’.?!…,:;]",'',x)  ) ))))  
    dataframe = dataframe.assign(**dict(zip([col,'num_extra'], zip(*dataframe[col].apply(lambda x: re.subn(r'[^a-zA-Z\s0-9]','',x)  ) ))))  
    dataframe = dataframe.assign(**dict(zip([col,'nm_all_tok','nm_pos_tok','nm_neg_tok','sm_pos_tok','sum_nrg_tok'], zip(*dataframe[col].apply(lambda x: return_tok_val(x) ) ))))
    dataframe['lex_tok'] = dataframe['nm_pos_tok'] + dataframe['nm_neg_tok']
    dataframe['total_lex_score'] = dataframe['sm_pos_tok'] + dataframe['sum_nrg_tok']
    dataframe['r_sco/tok'] = dataframe['total_lex_score']/dataframe['lex_tok']
    for col in dataframe.columns.difference(['clean_text','text','sentiment']):
        dataframe[col] += abs(dataframe[col].min())
    
    dataframe = dataframe.fillna(0)
    
    return dataframe

In [5]:
df_train = pd.read_csv(BASE_DIR + 'twitter-training-data.txt',sep='\t',names=['ID','sentiment','text']).drop('ID',axis=1)
df_dev = pd.read_csv(BASE_DIR + 'twitter-dev-data.txt',sep='\t',names=['ID','sentiment','text']).drop('ID',axis=1)
# df_train = pd.concat([df_train,df_dev]).reset_index(drop=True)
# del df_dev
df_test = pd.read_csv(BASE_DIR + 'twitter-test1.txt',sep='\t',names=['ID','sentiment','text']).drop('ID',axis=1)

In [10]:
df_train['sentiment'].value_counts()/len(df_train)

neutral     0.460823
positive    0.354462
negative    0.184715
Name: sentiment, dtype: float64

In [11]:
df_dev['sentiment'].value_counts()/len(df_dev)

neutral     0.4595
positive    0.3515
negative    0.1890
Name: sentiment, dtype: float64

In [12]:
df_test['sentiment'].value_counts()/len(df_test)

neutral     0.425942
positive    0.416313
negative    0.157746
Name: sentiment, dtype: float64

In [6]:
df_train['clean_text'] = df_train['text']
df_test['clean_text'] = df_test['text']

df_train = clean_data(df_train,'clean_text',pattern_binc,pattern_bdec,pattern_pos,pattern_neg,pattern_bad)
df_test = clean_data(df_test,'clean_text',pattern_binc,pattern_bdec,pattern_pos,pattern_neg,pattern_bad)

C:\Users\raghu\Anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
C:\Users\raghu\Anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [12]:
replace = {'positive': 0, 'negative': 1, 'neutral': 2}
df_train['sentiment'] = df_train['sentiment'].replace(replace)
df_test['sentiment'] = df_test['sentiment'].replace(replace)

In [13]:
tfidf = TfidfVectorizer(tokenizer=lambda x: x,preprocessor=lambda x: x,ngram_range=(1,3),min_df=10)
tfidf_vectorizer_vectors_train = tfidf.fit_transform(df_train.loc[:,'clean_text'].values)
tfidf_vectorizer_vectors_test = tfidf.transform(df_test.loc[:,'clean_text'].values)
pickle.dump(tfidf.vocabulary_, open(BASE_DIR + 'TFIDF.pkl', "wb"))
sleep(1)

train_x = tfidf_vectorizer_vectors_train.toarray()
test_x = tfidf_vectorizer_vectors_test.toarray()
train_y = df_train['sentiment'].values
test_y = df_test['sentiment'].values

In [14]:
train_x = np.concatenate((train_x,df_train[df_train.columns.difference(['sentiment', 'text', 'clean_text'])].values),axis=1)
test_x = np.concatenate((test_x,df_test[df_test.columns.difference(['sentiment', 'text', 'clean_text'])].values),axis=1)

In [19]:
model = MultinomialNB().fit(train_x,train_y)  ## 6396  and ## 6912
predicted = model.predict(train_x)
print(f1_score(train_y,predicted,average='macro'))
predicted_test = model.predict(test_x)
print(f1_score(test_y,predicted_test,average='macro'))

0.6218149693013134
0.6661988047388181


In [34]:
model = MultinomialNB().fit(train_x,train_y)  ## 6396  and ## 6912
predicted = model.predict(train_x)
print(f1_score(train_y,predicted,average='weighted'))
predicted_test = model.predict(test_x)
print(f1_score(test_y,predicted_test,average='weighted'))

0.697935994449718
0.6173130722383504


In [17]:
## Without is 66 and with is 65 something

df_test1 = pd.read_csv(BASE_DIR + 'twitter-test2.txt',sep='\t',names=['ID','sentiment','text']).drop('ID',axis=1)
df_test1['clean_text'] = df_test1['text']
df_test1 = clean_data(df_test1,'clean_text',pattern_binc,pattern_bdec,pattern_pos,pattern_neg,pattern_bad)
# df_test1 = df_test1.drop('num_exclaim',axis=1)
df_test1['sentiment'] = df_test1['sentiment'].replace(replace)
tfidf_vectorizer_vectors_test1 = tfidf.transform(df_test1.loc[:,'clean_text'].values)
test_x1 = tfidf_vectorizer_vectors_test1.toarray()
test_y1 = df_test1['sentiment'].values
test_x1 = np.concatenate((test_x1,df_test1[df_test1.columns.difference(['sentiment', 'text', 'clean_text'])].values),axis=1)
predicted_test1 = model.predict(test_x1)
print(f1_score(test_y1,predicted_test1,average='weighted'))

C:\Users\raghu\Anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


0.664229406489832


In [18]:
## Without  6304 , With 58 something
df_test1 = pd.read_csv(BASE_DIR + 'twitter-test3.txt',sep='\t',names=['ID','sentiment','text']).drop('ID',axis=1)
df_test1['clean_text'] = df_test1['text']
df_test1 = clean_data(df_test1,'clean_text',pattern_binc,pattern_bdec,pattern_pos,pattern_neg,pattern_bad)
# df_test1 = df_test1.drop('num_exclaim',axis=1)
df_test1['sentiment'] = df_test1['sentiment'].replace(replace)
tfidf_vectorizer_vectors_test1 = tfidf.transform(df_test1.loc[:,'clean_text'].values)
test_x1 = tfidf_vectorizer_vectors_test1.toarray()
test_y1 = df_test1['sentiment'].values
test_x1 = np.concatenate((test_x1,df_test1[df_test1.columns.difference(['sentiment', 'text', 'clean_text'])].values),axis=1)
predicted_test1 = model.predict(test_x1)
print(f1_score(test_y1,predicted_test1,average='weighted'))

C:\Users\raghu\Anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


0.6326952173891384


In [21]:
# df_train.to_pickle(BASE_DIR + 'train_data.pkl')
# df_test.to_pickle(BASE_DIR + 'test_data.pkl')

In [ ]:
# embeddings_dict = {}
# with open(BASE_DIR + "glove.6B.100d.txt", 'r', encoding='utf-8') as f:
#     for line in f:
#         values = line.split()
#         token = values[0]
#         vector = np.asarray(values[1:], "float32")
#         embeddings_dict[token] = vector

# from sklearn.feature_extraction.text import CountVectorizer
# count_vector = CountVectorizer(tokenizer=lambda x: x,preprocessor=lambda x: x)
# count_vector.fit(data_train['clean_text'])

# vocab = count_vector.vocabulary_

# min_list,max_list = [],[]
# while len(vocab) > 0:
#     min_word = min(vocab,key=vocab.get)
#     min_list.append(min_word)
#     vocab.pop(min_word)
#     if len(vocab) == 0:
#         break
#     max_word = max(vocab,key=vocab.get)
#     max_list.append(max_word)
#     vocab.pop(max_word)
    
# new_vocab = {}
# while len(new_vocab) != 4998:
#     word = max_list[-1]
#     vector = embeddings_dict.get(word)
#     if vector is None :
#         max_list.pop()
#     else:
#         new_vocab[word] = vector
#         max_list.pop()
#         if len(new_vocab) == 4998:
#             break
#     word = min_list[-1]
#     vector = embeddings_dict.get(word)
#     if vector is None:
#         min_list.pop()
#     else:
#         new_vocab[word] = vector
#         min_list.pop()
#         if len(new_vocab) == 4998:
#             break
            
# pickle.dump(new_vocab,open(BASE_DIR + 'LSTM_VOCAB.pkl', "wb"))